
<h1> DS200A Computer Vision Assignment</h1>

<h2>  Part Three: Training Models </h2>	

In [8]:
import pandas as pd
import numpy as np
import matplotlib as plt
import sklearn
from sklearn import model_selection
from sklearn.model_selection import GridSearchCV

In [9]:
def train_test_split(df):
    train_df, val_df = sklearn.model_selection.train_test_split(df, test_size=0.2)
    return train_df, val_df

#Split the data into a training set, and test set 
def accuracy(pred, actual):
    return np.mean(pred == actual)
# Calculate the accuracy percentage of the predicted values
def separate_features(df):
    feature_df = df.iloc[:,2:]
    labels_df = df.iloc[:,1]
    return feature_df, labels_df

In [10]:
training_data = pd.read_hdf('training_data.h5')

In [11]:
training_data.head()

,image,label,ft1,ft2,ft3,ft4,ft5,ft6,ft7,ft8,ft9,ft10,ft11,ft12,ft13,ft14,ft15,ft16,ft17
0,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",0,183.355727,176.750337,149.151581,2.426829,0.563561,0.540621,0.061394,0.024742,71,169.244714,0.566834,0.536543,164,0.771119,0.503230,2,0.000721
1,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",0,210.739822,189.280101,164.805242,2.179348,0.547861,0.540856,0.043255,0.019139,38,184.393053,0.596161,0.564449,184,0.623133,0.499767,3,0.000890
2,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",0,170.123787,147.788681,111.753798,2.381818,0.485925,0.527792,0.051072,0.034862,82,139.542571,0.523533,0.522927,165,0.547149,0.522024,2,0.001487
3,"[[[174, 190, 187], [173, 189, 186], [172, 188,...",0,152.159752,132.648316,77.917303,2.311765,0.443575,0.442804,0.059753,0.038761,126,118.502754,0.433735,0.430138,170,0.528061,0.476660,6,0.001969
4,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",0,147.397886,150.504790,86.252963,2.244318,0.479493,0.390340,0.046278,0.066373,93,130.929330,0.500365,0.399539,176,0.500000,0.500000,0,0.000000


<h3>  Train models using all of the following methods below. Be sure to drop the actual image column, and the encoding</h3>	Take note of the differences in accuracy, and methods.


In [12]:
train, val = train_test_split(training_data)
X_train, y_train = separate_features(train)
X_train.reset_index().drop(columns=['index'])
y_train.reset_index().drop(columns=['index'])
y_train = y_train.astype('int')

X_val, y_val = separate_features(val)
X_val.reset_index().drop(columns=['index'])
y_val.reset_index().drop(columns=['index'])
y_val = y_val.astype('int')

In [13]:
import warnings
warnings.filterwarnings("ignore")

def cross_val(model, k, feature_data, label_data):
    
    kf = sklearn.model_selection.KFold(n_splits = k)
    kf.get_n_splits(feature_data)
    
    accuracy_scores = np.empty([k, 1])
    model_params = [{} for i in range(feature_data.shape[1])]
        
    for i, (train_index, test_index) in enumerate(kf.split(feature_data)):
        model.fit(feature_data.iloc[train_index], label_data.iloc[train_index])
        label_predict = model.predict(feature_data.iloc[test_index])
        accuracy_scores[i] = accuracy(label_data.iloc[test_index], label_predict)
        model_params[i] = model.get_params()
        print(accuracy_scores[i])
    
    return model_params, accuracy_scores

Logistic Regression

In [14]:
from sklearn.linear_model import LogisticRegression

lreg = LogisticRegression(random_state=0, penalty='l2', C=1)
params = {'C': [1e-1, 1, 10, 100]}
grid_lreg = GridSearchCV(lreg, params, cv=5)
grid_lreg.fit(X_train, y_train);

K-nearest Neighbors

In [15]:
from sklearn.neighbors import KNeighborsClassifier

params = {'n_neighbors': [1,2,4,8], 'algorithm': ['ball_tree', 'kd_tree']}
neigh = KNeighborsClassifier()
grid_neigh = GridSearchCV(neigh, params, cv=5)
grid_neigh.fit(X_train, y_train);

Random Forest

In [16]:
from sklearn.ensemble import RandomForestClassifier

rand_forest = RandomForestClassifier(
    random_state=0, 
    n_estimators=25, 
    max_depth=5, 
    bootstrap=True
)
params = {'n_estimators': [25,75,150], 'max_depth': [10,20,80]}
grid_rand_forest = GridSearchCV(rand_forest, params, cv=5)
grid_rand_forest.fit(X_train, y_train);

Support Vector Machine

In [17]:
from sklearn.multiclass import OneVsOneClassifier
from sklearn.svm import LinearSVC

params = {'C': [1e-1, 1, 10, 100], 'loss': ['hinge', 'squared_hinge']}
one_vs_one_svm = OneVsOneClassifier(LinearSVC(random_state=0, penalty='l2', C=100))
one_vs_one_svm.fit(X_train, y_train);
# grid_ovo_svm = GridSearchCV(one_vs_one_svm, params, cv=5)
# grid_ovo_svm.fit(X_train, y_train);

# from sklearn.svm import LinearSVC
# params = {'C': [1e-1, 1, 10, 100], 'loss': ['hinge', 'squared_hinge']}
# grid_svm = GridSearchCV(LinearSVC(), params, cv=5)
# grid_svm.fit(X_train, y_train);

In [19]:
from sklearn.multiclass import OneVsRestClassifier
# params = {'C': [1e-1, 1, 10, 100], 'loss': ['hinge', 'squared_hinge']}
one_vs_rest_svm = OneVsRestClassifier(LinearSVC(random_state=0, penalty='l2', C=.1))
# grid_ovr_svm = GridSearchCV(one_vs_rest_svm, params, cv=5)
one_vs_rest_svm.fit(X_train, y_train);

In [20]:
models = [grid_lreg, grid_neigh, grid_rand_forest, one_vs_one_svm, one_vs_rest_svm]
names = ['Logistic Regression', 'K-NN', 'Random Forest', 'One v One SVM', 'One v Rest SVM']
max_name_length = len(max(names, key=len))

for model, name in zip(models, names):
    y_hat = model.predict(X_val)
    acc = accuracy(y_hat, y_val)
    print('{:19s} Accuracy: {}'.format(name, acc))

Logistic Regression Accuracy: 0.3654485049833887
K-NN                Accuracy: 0.24584717607973422
Random Forest       Accuracy: 0.3920265780730897
One v One SVM       Accuracy: 0.22259136212624583
One v Rest SVM      Accuracy: 0.12292358803986711
